### Pretrained Feature Extractor

In [8]:
import sys
sys.path.insert(0, '../')
from config import *
from dataset import MNIST,CIFAR10, MNIST_SUB

In [51]:
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models import resnet50, ResNet50_Weights

weights = ResNet50_Weights.DEFAULT
# I use ResNet for now, but can be changed to other model
# Will use ViT in the future
model = resnet50(weights=weights)

# model.eval()
nodes, _ = get_graph_node_names(model)

In [9]:
ind_idx = [0,2,3,6,8]
ood_idx = [1,7]
# TODO: Use classifier on 1,4,5,7,9, check wass loss
# test_idx = [4,5,9]
mnist_dset_dict = MNIST_SUB(batch_size=128, val_batch_size=64, idx_ind=ind_idx, idx_ood=ood_idx, shuffle=True)

In [10]:
ind_train_loader = mnist_dset_dict['train_set_ind_loader']
ind_val_loader = mnist_dset_dict['val_set_ind_loader']

In [52]:
# print(nodes)
preprocess = weights.transforms()
# Bilinear interpolation is important !
print(preprocess)

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [22]:
x, y = next(iter(ind_train_loader))
# Temporary test
x = torch.repeat_interleave(x, 3, 1) # This is needed just for testing; for CIFAR, ImageNet this is not needed.
# print(preprocess(x))

In [49]:
# CAUTION: get rid of the last three layers !
# print(nodes)
# print(nodes[:-3])
valid = nodes[:-3]
feat_extractor = create_feature_extractor(
	model, return_nodes=valid)

In [50]:
feat_extractor.requires_grad_(False)
out = feat_extractor(x)
print(out[valid[-1]].shape)

torch.Size([128, 2048, 1, 1])
